In [66]:
import os
import subprocess
import duckdb
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

# --- Settings ---
seasons = [2019, 2020, 2021, 2022, 2023, 2024]
base_url = 'https://github.com/nflverse/nflverse-data/releases/download/player_stats/player_stats_{year}.parquet'
local_dir = "data/raw/nflfastr"
os.makedirs(local_dir, exist_ok=True)


# --- Download with curl to ensure redirects handled ---
def download_file_curl(url, output_path):
    if os.path.exists(output_path):
        print(f"✅ Already exists: {output_path}")
        return
    print(f"⬇️ Downloading: {url}")
    subprocess.run(['curl', '-L', url, '-o', output_path], check=True)
    print(f"✅ Saved: {output_path}")


# --- Download files for each season ---
files = []
for season in seasons:
    url = base_url.format(year=season)
    local_path = os.path.join(local_dir, f"player_stats_{season}.parquet")
    download_file_curl(url, local_path)
    files.append(local_path)


# --- Ingest with DuckDB ---
dfs = []
for file in files:
    if os.path.getsize(file) < 1000:
        raise ValueError(f"❌ File {file} is too small to be valid. Aborting.")
    print(f"📥 Processing {file} ...")
    query = f"""
        SELECT 
            season, 
            week, 
            player_name AS name, 
            passing_yards, 
            rushing_yards, 
            receiving_yards, 
            passing_tds + rushing_tds + receiving_tds AS touchdowns, 
            fantasy_points_ppr AS fantasy_points 
        FROM read_parquet('{file}')
    """
    df = duckdb.query(query).to_df()
    dfs.append(df)

combined_df = pd.concat(dfs)
print(f"✅ Combined records: {len(combined_df)}")


# --- Map player_id from players table ---
players_df = pd.read_sql('SELECT player_id, name FROM players', engine)
merged_df = combined_df.merge(players_df, how='left', on='name')

# --- Report unmatched players ---
unmatched = merged_df[merged_df['player_id'].isnull()]
if not unmatched.empty:
    print("⚠️ Unmatched players (not in your `players` table):")
    print(unmatched['name'].dropna().unique())

# --- Filter out unmatched for DB insert ---
final_df = merged_df[merged_df['player_id'].notnull()]

# --- Prepare for Postgres insert ---
final_df = final_df[['player_id', 'season', 'week', 'passing_yards', 'rushing_yards', 'receiving_yards', 'touchdowns', 'fantasy_points']]

# --- Insert into Postgres ---
final_df.to_sql('weekly_stats', engine, if_exists='append', index=False)
print(f"✅ {len(final_df)} rows successfully ingested into `weekly_stats`.")


⬇️ Downloading: https://github.com/nflverse/nflverse-data/releases/download/player_stats/player_stats_2019.parquet


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  313k  100  313k    0     0   225k      0  0:00:01  0:00:01 --:--:-- 1585k


✅ Saved: data/raw/nflfastr/player_stats_2019.parquet
⬇️ Downloading: https://github.com/nflverse/nflverse-data/releases/download/player_stats/player_stats_2020.parquet


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  320k  100  320k    0     0   317k      0  0:00:01  0:00:01 --:--:--  317k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

✅ Saved: data/raw/nflfastr/player_stats_2020.parquet
⬇️ Downloading: https://github.com/nflverse/nflverse-data/releases/download/player_stats/player_stats_2021.parquet


...

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  325k  100  325k    0     0   432k      0 --:--:-- --:--:-- --:--:-- 5418k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

✅ Saved: data/raw/nflfastr/player_stats_2023.parquet
⬇️ Downloading: https://github.com/nflverse/nflverse-data/releases/download/player_stats/player_stats_2024.parquet


  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


✅ Saved: data/raw/nflfastr/player_stats_2024.parquet
📥 Processing data/raw/nflfastr/player_stats_2019.parquet ...
📥 Processing data/raw/nflfastr/player_stats_2020.parquet ...
📥 Processing data/raw/nflfastr/player_stats_2021.parquet ...
📥 Processing data/raw/nflfastr/player_stats_2022.parquet ...
📥 Processing data/raw/nflfastr/player_stats_2023.parquet ...
📥 Processing data/raw/nflfastr/player_stats_2024.parquet ...
✅ Combined records: 33287
⚠️ Unmatched players (not in your `players` table):
['T.Brady' 'D.Brees' 'J.McCown' ... 'M.Penix' 'R.Odunze' 'M.Corley']
✅ 0 rows successfully ingested into `weekly_stats`.


100  323k  100  323k    0     0   591k      0 --:--:-- --:--:-- --:--:--  591k


In [72]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

response = requests.get('https://api.sleeper.app/v1/players/nfl')
players_data = response.json()
players_df = pd.DataFrame.from_dict(players_data, orient='index')

depth_chart_df = players_df[['full_name', 'position', 'depth_chart_position', 'team']].copy()
depth_chart_df = depth_chart_df.rename(columns={
    'full_name': 'name',
    'depth_chart_position': 'depth_position'
})
depth_chart_df = depth_chart_df[depth_chart_df['name'].notnull() & depth_chart_df['position'].notnull()]

# Link team ID from existing teams table
teams_query = pd.read_sql('SELECT team_id, abbreviation FROM teams', engine)
depth_chart_df = depth_chart_df.merge(teams_query, how='left', left_on='team', right_on='abbreviation')

# Link player ID from existing players table
players_query = pd.read_sql('SELECT player_id, name FROM players', engine)
depth_chart_df = depth_chart_df.merge(players_query, how='left', on='name')

# Drop rows where we are missing player_id or team_id because those are required for our schema
final_df = depth_chart_df[['player_id', 'team_id', 'position', 'depth_position']].copy()

# Drop rows where player_id or team_id is string 'None'
final_df = final_df[(final_df['player_id'] != 'None') & (final_df['team_id'] != 'None')]

# Drop rows where player_id or team_id is NaN
final_df = final_df.dropna(subset=['player_id', 'team_id'])

# Ensure player_id and team_id are integers
final_df['player_id'] = final_df['player_id'].astype(int)
final_df['team_id'] = final_df['team_id'].astype(int)

# Clean depth_position
final_df['depth_position'] = final_df['depth_position'].replace('None', pd.NA)
final_df['depth_position'] = final_df['depth_position'].fillna(0)
final_df['depth_position'] = final_df['depth_position'].astype(str)

final_df.to_sql('depth_chart', engine, if_exists='replace', index=False)
print("✅ Depth chart ingested into PostgreSQL.")


✅ Depth chart ingested into PostgreSQL.


In [73]:
final_df.head()

,player_id,team_id,position,depth_position
6,6,14,CB,NB
8,8,30,RB,0
12,12,1,CB,0
14,14,9,K,K
18,18,29,DE,LOLB


In [74]:
final_df.shape

(3406, 4)

In [71]:
final_df['team_id'].isnull().sum()

np.int64(0)

In [83]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load env vars
load_dotenv()

# DB connection
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

# --- Fetch Teams ---
teams_data = [
    {"name": "Arizona Cardinals", "abbreviation": "ARI"},
    {"name": "Atlanta Falcons", "abbreviation": "ATL"},
    {"name": "Baltimore Ravens", "abbreviation": "BAL"},
    {"name": "Buffalo Bills", "abbreviation": "BUF"},
    {"name": "Carolina Panthers", "abbreviation": "CAR"},
    {"name": "Chicago Bears", "abbreviation": "CHI"},
    {"name": "Cincinnati Bengals", "abbreviation": "CIN"},
    {"name": "Cleveland Browns", "abbreviation": "CLE"},
    {"name": "Dallas Cowboys", "abbreviation": "DAL"},
    {"name": "Denver Broncos", "abbreviation": "DEN"},
    {"name": "Detroit Lions", "abbreviation": "DET"},
    {"name": "Green Bay Packers", "abbreviation": "GB"},
    {"name": "Houston Texans", "abbreviation": "HOU"},
    {"name": "Indianapolis Colts", "abbreviation": "IND"},
    {"name": "Jacksonville Jaguars", "abbreviation": "JAX"},
    {"name": "Kansas City Chiefs", "abbreviation": "KC"},
    {"name": "Las Vegas Raiders", "abbreviation": "LV"},
    {"name": "Los Angeles Chargers", "abbreviation": "LAC"},
    {"name": "Los Angeles Rams", "abbreviation": "LAR"},
    {"name": "Miami Dolphins", "abbreviation": "MIA"},
    {"name": "Minnesota Vikings", "abbreviation": "MIN"},
    {"name": "New England Patriots", "abbreviation": "NE"},
    {"name": "New Orleans Saints", "abbreviation": "NO"},
    {"name": "New York Giants", "abbreviation": "NYG"},
    {"name": "New York Jets", "abbreviation": "NYJ"},
    {"name": "Philadelphia Eagles", "abbreviation": "PHI"},
    {"name": "Pittsburgh Steelers", "abbreviation": "PIT"},
    {"name": "San Francisco 49ers", "abbreviation": "SF"},
    {"name": "Seattle Seahawks", "abbreviation": "SEA"},
    {"name": "Tampa Bay Buccaneers", "abbreviation": "TB"},
    {"name": "Tennessee Titans", "abbreviation": "TEN"},
    {"name": "Washington Commanders", "abbreviation": "WAS"}
]
teams_df = pd.DataFrame(teams_data).reset_index().rename(columns = {'index':'team_id'})
teams_df.to_sql('teams', engine, if_exists='replace', index=False)
print("✅ Teams ingested into PostgreSQL.")

✅ Teams ingested into PostgreSQL.


In [77]:
teams_df.reset_index()

,index,name,abbreviation
0,0,Arizona Cardinals,ARI
1,1,Atlanta Falcons,ATL
2,2,Baltimore Ravens,BAL
3,3,Buffalo Bills,BUF
4,4,Carolina Panthers,CAR
5,5,Chicago Bears,CHI
6,6,Cincinnati Bengals,CIN
7,7,Cleveland Browns,CLE
8,8,Dallas Cowboys,DAL
9,9,Denver Broncos,DEN


In [84]:
print("Fetching players from Sleeper API...")
response = requests.get('https://api.sleeper.app/v1/players/nfl')
players_data = response.json()

df = pd.DataFrame.from_dict(players_data, orient='index')

df = df[['full_name', 'position', 'team', 'birth_date']]
df = df.rename(columns={
    'full_name': 'name',
    'birth_date': 'birthdate'
})

# Filter out entries without a name (defensive team slots, placeholders)
df = df[df['name'].notnull() & df['position'].notnull()]

# Handle team ID mapping via abbreviation
teams_query = pd.read_sql('SELECT team_id, abbreviation FROM teams', engine)
df = df.merge(teams_query, how='left', left_on='team', right_on='abbreviation')

df = df[['name', 'position', 'team_id', 'birthdate']]

Fetching players from Sleeper API...


In [86]:
df.reset_index().rename(columns = {'index': 'player_id'})

,player_id,name,position,team_id,birthdate
0,0,Ellis Richardson,TE,NaN,1995-02-12
1,1,Nick Amoah,OL,NaN,None
2,2,Malkelm Morrison,CB,NaN,None
3,3,Carl Tucker,TE,NaN,1997-02-06
4,4,C.J. Mosley,LB,NaN,1992-06-19
...,...,...,...,...,...
17321,17321,Quincy McDuffie,WR,NaN,1990-09-24
17322,17322,Kwayde Miller,OT,NaN,1993-12-11
17323,17323,Niles Paul,TE,NaN,1989-08-09
17324,17324,Alvin Bailey,G,NaN,1991-08-26
